In [1]:
import socket
import threading

HOST = "127.0.0.1"
PORT = 9999

def start_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind((HOST, PORT))
    server.listen(1)
    print(f"Socket server listening on {HOST}:{PORT}")
    conn, addr = server.accept()
    print(f"Client connected from {addr}")

    # Keep connection open; producer will send data through this socket
    while True:
        data = conn.recv(1024)
        if not data:
            break

    conn.close()
    server.close()

threading.Thread(target=start_server, daemon=True).start()


Socket server listening on 127.0.0.1:9999


In [2]:
import time
import socket

def stream_csv_to_socket(csv_path='/content/data.csv', host="127.0.0.1", port=9999, delay=0.5, skip_header=True):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, port))
    with open(csv_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if skip_header and i == 0:
                continue
            # ensure each line ends with newline
            s.sendall((line.strip() + "\n").encode("utf-8"))
            time.sleep(delay)  # simulate streaming
    s.shutdown(socket.SHUT_WR)
    s.close()

# Start streaming in background so you can run Spark consumer next
threading.Thread(target=stream_csv_to_socket, args=("/content/data.csv",), daemon=True).start()


Client connected from ('127.0.0.1', 32862)


In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext(appName="StreamingRemoveHeader")
ssc = StreamingContext(sc, 10)  # batch interval = 10 seconds


/usr/local/lib/python3.12/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [4]:
lines = ssc.socketTextStream("127.0.0.1", 9999)


In [5]:
header = "InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country"

# Remove header line from each batch
# filtered_lines = lines.filter(lambda line: line != header)

filtered_lines = lines.filter(lambda line: "InvoiceNo" not in line)


In [6]:
def parse_line(line):
    fields = line.split(",")
    try:
        quantity = int(fields[3])
        unit_price = float(fields[5])
        customer_id = fields[6]
        total_price = quantity * unit_price
        return (customer_id, total_price)
    except:
        return ("Invalid", 0.0)

parsed_stream = filtered_lines.map(parse_line)


In [7]:
customer_totals = parsed_stream.reduceByKey(lambda a, b: a + b)
customer_totals.pprint()


In [8]:
ssc.start()
ssc.awaitTermination()


-------------------------------------------
Time: 2026-01-03 20:04:20
-------------------------------------------

-------------------------------------------
Time: 2026-01-03 20:04:30
-------------------------------------------

-------------------------------------------
Time: 2026-01-03 20:04:40
-------------------------------------------



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 